# **$LiH$ Ground State with Qiskit**

By Martin Vuelta ([@zodiacfireworks](https://github.com/zodiacfireworks))

## **Preliminaries**

### *Check machine capabilities*

In [ ]:
!free -h

In [ ]:
!lscpu | egrep 'Model name|Socket|Thread|NUMA|CPU\(s\)'

### *Install dependecies*

In [ ]:
!pip install --upgrade qiskit dill

from IPython.display import clear_output

clear_output()

### *Qiskit imports*

In [ ]:
import pickle
from datetime import datetime
from functools import partial

import dill
import numpy as np
import pandas
import qiskit.tools.jupyter
from qiskit import IBMQ, Aer, BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, ExactEigensolver, NumPyEigensolver
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.components.variational_forms import RY, RYRZ, SwapRZ
from qiskit.aqua.operators import WeightedPauliOperator, Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit.library import EfficientSU2
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer import QasmSimulator, noise
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import (
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)
from qiskit.providers.ibmq import least_busy

The qiskit summary

In [ ]:
%qiskit_version_table

Loads the IBMQ account, please run just once!

In [ ]:
# token = "put your token here!"
# IBMQ.save_account(token, overwrite=True)

In [ ]:
provider = IBMQ.load_account()

### *Plot settings*

In [ ]:
from distutils.spawn import find_executable

import seaborn
from matplotlib import pyplot as plt
from matplotlib import rc

seaborn.set(context="notebook", style="darkgrid")
plt.ioff()

rem = 16
rc("lines", linewidth=1)
rc("font", family="serif")
rc("font", size=rem)
rc("axes", titlepad=1.500 * rem)
rc("axes", titlesize=1.728 * rem)
rc("axes", labelsize=1.200 * rem)
rc("legend", fontsize=1.000 * rem)
rc("xtick", labelsize=0.833 * rem)
rc("ytick", labelsize=0.833 * rem)

if find_executable("latex"):
    rc("text", usetex=True)

material_palette = {
    -1: "#212121",
    0: "#F44336",
    1: "#E91E63",
    2: "#9C27B0",
    3: "#673AB7",
    4: "#3F51B5",
    5: "#2196F3",
    6: "#03A9F4",
    7: "#00BCD4",
    8: "#009688",
    9: "#4CAF50",
    10: "#8BC34A",
    11: "#CDDC39",
    12: "#FFEB3B",
    13: "#FFC107",
    14: "#FF9800",
    15: "#FF5722",
}

In [ ]:
%matplotlib inline

### *Prevent the warnings sea*

In [ ]:
import warnings

warnings.simplefilter("ignore")

## **Simulation in action!**

### *Prepare the qubit operations*

In [ ]:
def compute_LiH_qubitOp(map_type, inter_dist, basis="sto3g"):
    driver = PySCFDriver(
        atom=f"Li .0 .0 .0; H .0 .0 {inter_dist}",
        unit=UnitsType.ANGSTROM,
        charge=0,
        spin=0,
        basis=basis,
    )

    molecule = driver.run()

    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals

    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2

    freeze_list = [0]
    remove_list = [-3, -2]

    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]

    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list) for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]

    energy_shift = 0.0
    qubit_reduction = True if map_type == "parity" else False

    ferOp = FermionicOperator(h1=h1, h2=h2)

    if len(freeze_list) > 0:
        ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
        num_spin_orbitals -= len(freeze_list)
        num_particles -= len(freeze_list)

    if len(remove_list) > 0:
        ferOp = ferOp.fermion_mode_elimination(remove_list)
        num_spin_orbitals -= len(remove_list)

    qubitOp = ferOp.mapping(map_type=map_type)
    qubitOp = (
        Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
        if qubit_reduction
        else qubitOp
    )

    qubitOp.chop(10 ** -10)

    return {
        "molecule": molecule,
        "qubitOp": qubitOp,
        "num_spin_orbitals": num_spin_orbitals,
        "num_particles": num_particles,
        "qubit_reduction": qubit_reduction,
        "energy_shift": energy_shift + nuclear_repulsion_energy,
    }

### *Prepere basic experiment conditions*

In [ ]:
map_types = ["parity", "bravyi_kitaev", "jordan_wigner"]
distances = np.linspace(0.5, 3.5, 31)

Since qubit operations dependes on qubit mapping and interatomic distance, I setup a settings map as follows

In [ ]:
timer = datetime.now()

# Effective computation of qubitOp for (distance, map_type)
# combinations
settings_tree = {}
combination_counter = 0

for distance in distances:
    for map_type in map_types:
        key = (
            distance,
            map_type,
        )

        if key not in settings_tree:
            settings_tree[key] = compute_LiH_qubitOp(
                map_type=map_type, inter_dist=distance, basis="sto3g"
            )
            combination_counter = combination_counter + 1

timer = datetime.now() - timer

print("Settings computed!")
print(f"{combination_counter} configuration settings computed in {timer}")

Now save the object generated in pickle format in order to reuse them and avoid the pain to run settings computation again

In [ ]:
file = open("settings_tree.pkl", "wb")
dill.dump(settings_tree, file)
file.close()

Run the following cell in order to load `settings_tree` from pickle file instead of recomputting it again

In [ ]:
# file = open("settings_tree.pkl", "rb")
# settings_tree = dill.load(file)
# file.close()

### *Look for exact solutions*

In [ ]:
def exact_solver(qubitOp):
    solver = NumPyEigensolver(qubitOp)

    return solver.run()

In [ ]:
timer = datetime.now()

# Effective computation of solutions using NumPyEigensolver (aka ExactEigensolver)
# for (distance, map_type) combinations in settings_tree
exact_energies = {}
combination_counter = 0

for key in settings_tree:
    if key not in exact_energies:
        settings = settings_tree[key]
        qubitOp = settings["qubitOp"]

        # Exact energy computation
        solver_timer = datetime.now()
        exact_energies[key] = exact_solver(qubitOp)
        solver_timer = datetime.now() - solver_timer

        exact_energies[key] = dict(exact_energies[key])
        exact_energies[key].update({"execution_time": solver_timer})
        combination_counter = combination_counter + 1

timer = datetime.now() - timer

print("Exact Energies computed!")
print(f"{combination_counter} energies computed in {timer}")

Now save the object generated in pickle format in order to reuse them and avoid the pain to run exact energy computation again

In [ ]:
file = open("exact_energies.pkl", "wb")
dill.dump(exact_energies, file)
file.close()

Run the following cell in order to load `settings_tree` from pickle file instead of recomputting it again

In [ ]:
# file = open("exact_energies.pkl", "rb")
# exact_energies = dill.load(file)
# file.close()

#### Showing data as table

In [ ]:
data = {"distance": distances}

for map_type in map_types:
    data[map_type] = []

    for key in exact_energies:
        if map_type in key:
            # Get the interatomic distance
            distance, _ = key

            # Get the energy shift
            settings = settings_tree[key]
            shift = settings["energy_shift"]

            energy = exact_energies[key]
            energy = energy["eigenvalues"][0].real
            energy = energy + shift

            data[map_type].append((distance, energy))

    # ensure sorting
    data[map_type] = sorted(data[map_type], key=lambda x: x[0])
    _, data[map_type] = zip(*data[map_type])

data = pandas.DataFrame(data=data)
data

Save the data to CSV

In [ ]:
data.to_csv("exact_energies.csv", index=False)

#### Plot for exact solutions

In [ ]:
ratio = 16.0 / 9.0
width = 16
height = width / ratio

fig = plt.figure(1, figsize=(width, height), frameon=False)
axs = fig.add_subplot("111")

x = data["distance"]

for i, map_type in enumerate(map_types, 1):
    y = data[map_type]
    label = r"$\mathtt{{{0}}}$".format(map_type.replace("_", "\_"))
    axs.plot(x, y, label=label, color=material_palette[(i + 2) ** 2 % 16])

axs.set_xlim(0, 4)
axs.set_xlabel("$\mathrm{Atomic\ distance\ (\AA)}$")

axs.set_ylim(-8, -7)
axs.set_ylabel("Energy")
axs.set_yticks([y for y in np.arange(-8, -6.9, 0.2)])

axs.set_title("$\mathrm{Energy\ for\ LiH}$")

axs.legend(loc=1, title="$\mathbf{Qubit\ mappings}$")
axs.grid(linestyle="-")
fig.tight_layout()
plt.show()

### *Looking for quantum solver*

#### Useful functions and variables

In [ ]:
def get_device(provider, chip_name=None, min_qubits=0):
    """Return device with name `chip_name` from provider.
    
    If `chip_name` is not provided then return the least busy device.
    """

    if chip_name:
        return provider.get_backend(chip_name)

    return least_busy(
        provider.backends(
            filters=lambda x: (
                x.configuration().n_qubits >= min_qubits
                and not x.configuration().simulator
                and x.status().operational == True
            )
        )
    )

In [ ]:
def get_backend(kind, device=None, **kwargs):
    """Return the QuantumInstance backend,
    
    If `kind='noisy'` then `device` is required and `kwargs` must contain
    `shots` key.
    """

    if kind == "sv":
        return Aer.get_backend("statevector_simulator")

    if kind == "qasm":
        return Aer.get_backend("qasm_simulator")

    if kind == "noisy":
        coupling_map = device.configuration().coupling_map
        noise_model = noise.device.basic_device_noise_model(device.properties())
        basis_gates = noise_model.basis_gates

        return QuantumInstance(
            backend=Aer.get_backend("qasm_simulator"),
            noise_model=noise_model,
            coupling_map=coupling_map,
            measurement_error_mitigation_cls=CompleteMeasFitter,
            cals_matrix_refresh_period=30,
            basis_gates=noise_model.basis_gates,
            **kwargs,
        )

    raise ValueError(f"`kind` must be 'sv', 'qasm' or 'noisy', not {kind}")

In [ ]:
def get_optimizer(kind, iterations=100, **kwargs):
    if kind == "cobyla":
        return COBYLA(maxiter=iterations, **kwargs)

    if kind == "l_bfgs_b":
        return L_BFGS_B(maxiter=iterations, **kwargs)

    if kind == "slsqp":
        return SLSQP(maxiter=iterations, **kwargs)

    if kind == "spsa":
        return SPSA(max_trials=iterations, **kwargs)

    raise ValueError(
        "`kind` mist be one `cobyla`, `l_bfgs_b`, `slsqp` or `spsa`, " f" not {kind}"
    )

In [ ]:
def init_state(kind, qubitOp=None, **kwargs):
    """Return the initial state based on `kind` parameter.
    
    If `kind='hf'`, `kwargs` must contain `num_orbitals`, `num_particles`,
    `map_type` and `qubit_reduction` keys for a correct HartreeFock state 
    initialization.
    """

    if kind == "zero":
        return Zero(qubitOp.num_qubits)

    if kind == "hf":
        qubit_mapping = kwargs.get("qubit_mapping", None)

        if qubit_mapping and qubit_mapping != "parity":
            kwargs.update({"two_qubit_reduction": False})

        return HartreeFock(
            **kwargs
            # num_orbitals=settings["qubitOp"].num_qubits,
            # num_particles=settings["num_particles"],
            # qubit_mapping=map_type,
            # two_qubit_reduction=settings["qubit_reduction"],
        )

    raise ValueError(f"`kind` must be 'zero' or 'hf', not {kind}")

In [ ]:
def variational_form(kind, initial_state, qubitOp=None, **kwargs):
    """Return variational form given it's kind.
    
    If `kind="uccsd"`, `kwargs` must contains the folowing keys:
        - `num_orbitals`
        - `num_particles`
        - `qubit_mapping`
        - `two_qubit_reduction`
    """
    if kind == "uccsd":
        return UCCSD(
            **kwargs,
            initial_state=initial_state,
            excitation_type="s",
            method_singles="beta",
            reps=2,
            # num_orbitals=settings["num_spin_orbitals"],
            # num_particles=settings["num_particles"],
            # qubit_mapping=map_type,
            # two_qubit_reduction=settings["qubit_reduction"],
        )

    if kind == "ry":
        return RY(qubitOp.num_qubits, initial_state=initial_state)

    if kind == "ryrz":
        var_form = RYRZ(qubitOp.num_qubits, initial_state=initial_state)

    if kind == "swaprz":
        var_form = SwapRZ(qubitOp.num_qubits, initial_state=initial_state)

    if kind == "esu2":
        var_form = EfficientSU2(
            qubitOp.num_qubits, initial_state=initial_state, entanglement="linear",
        )

    raise ValueError(
        "`kind` mist be one `uccsd`, `ry`, `ryrz`, `swaprz` " f"or `esu2`, not {kind}"
    )

In [ ]:
def aproximate_solver(qubitOp, form, backend, optimizer, **kwargs):
    solver = VQE(qubitOp, form, optimizer, **kwargs)
    return solver.run(backend)

#### Experiments with IMBQ

This was initialy made for HartreFock initial state, UCCSD variational form and COBYLA optimizer with noisy backend, but now you can vary parasm to get many experiments :D

In [ ]:
device = get_device(provider, min_qubits=6)
device

The experiment starts here! Read comments gto change execution params

In [ ]:
# -*- BEGIN PARAMS -*-
# Change the following params to change experiment setup

# backend_kind must be one of "sv", "qasm", "noisy"
backend_kind = "noisy"
shots = 100

# initial_state_kind must be one of "zero", "hf"
initial_state_kind = "hf"

# variational_form_kind must be one of "uccsd", "ry", "ryrz", "swaprz", "esu2":
variational_form_kind = "uccsd"

# optimizer_kind must be one of "cobyla", "l_bfgs_b", "slsqp", "spsa"
optimizer_kind = "slsqp"
iterations = 100

# -*- END PARAMS -*-

timer = datetime.now()
experiment_settings = (
    backend_kind,
    shots,
    initial_state_kind,
    variational_form_kind,
    optimizer_kind,
    iterations,
)
backend = get_backend(backend_kind, device, shots=shots)

# Effective computation of solutions using Approximate solver (see
# experiment_settings) for (distance, map_type) combinations in settings_tree
approximate_energies = {}
intermediate_results = {}
combination_counter = 0

for key in settings_tree:
    if key not in approximate_energies:
        settings = settings_tree[key]
        qubitOp = settings["qubitOp"]

        state = init_state(
            kind=initial_state_kind,
            qubitOp=qubitOp,
            # **kwargs,
            num_orbitals=settings["num_spin_orbitals"],
            num_particles=settings["num_particles"],
            qubit_mapping=map_type,
            two_qubit_reduction=settings["qubit_reduction"],
        )

        form = variational_form(
            variational_form_kind,
            state,
            qubitOp,
            # **kwargs
            num_orbitals=settings["num_spin_orbitals"],
            num_particles=settings["num_particles"],
            qubit_mapping=map_type,
            two_qubit_reduction=settings["qubit_reduction"],
        )

        optimizer = get_optimizer(optimizer_kind, iterations=iterations)

        intermediate_results[key] = {
            "counts": [],
            "values": [],
            "params": [],
            "deviation": [],
        }

        def callback(eval_count, parameters, mean, std):
            intermediate_results[key]["counts"].append(eval_count)
            intermediate_results[key]["values"].append(mean)
            intermediate_results[key]["params"].append(parameters)
            intermediate_results[key]["deviation"].append(std)

        # Approximate energy computation
        solver_timer = datetime.now()
        approximate_energies[key] = aproximate_solver(
            qubitOp,
            form,
            backend,
            optimizer,
            # **kwargs,
            max_evals_grouped=10,
            callback=callback,
        )
        solver_timer = datetime.now() - solver_timer

        approximate_energies[key] = dict(approximate_energies[key])
        approximate_energies[key].update({"execution_time": solver_timer})

        combination_counter = combination_counter + 1

        print(key, solver_timer)

timer = datetime.now() - timer

print("Exact Energies computed!")
print(f"{combination_counter} energies computed in {timer}")

Select a file name that reflects the settings of the experiment

In [ ]:
file_name = "__".join([str(item) for item in experiment_settings])
file_name

Now save the object generated in pickle format in order to reuse them and avoid the pain to run exact energy computation again

In [ ]:
file = open(f"{file_name}.pkl", "wb")
dill.dump((experiment_settings, approximate_energies, intermediate_results), file)
file.close()

Run the following cell in order to load `experiment_settings`, `approximate_energies`, `intermediate_results` from pickle file instead of recomputting it again

In [ ]:
# file_name = ...
# file = open(f"{file_name}.pkl", "rb")
# experiment_settings, approximate_energies, intermediate_results = dill.load(file)
# file.close()

#### Showing data as table

In [ ]:
approximate_data = {"distance": distances}

for map_type in map_types:
    approximate_data[map_type] = []

    for key in approximate_energies:
        if map_type in key:
            # Get the interatomic distance
            distance, _ = key

            # Get the energy shift
            settings = settings_tree[key]
            shift = settings["energy_shift"]

            energy = approximate_energies[key]
            energy = energy["eigenvalue"].real
            energy = energy + shift

            approximate_data[map_type].append((distance, energy))

    # ensure sorting
    approximate_data[map_type] = sorted(approximate_data[map_type], key=lambda x: x[0])
    _, approximate_data[map_type] = zip(*approximate_data[map_type])

approximate_data = pandas.DataFrame(data=approximate_data)
approximate_data

Save the data to CSV

In [ ]:
approximate_data.to_csv(f"{file_name}.csv", index=False)

In [ ]:
ratio = 16.0 / 9.0
width = 16
height = width / ratio

fig = plt.figure(1, figsize=(width, height), frameon=False)
axs = fig.add_subplot("111")

x = data["distance"]

for i, map_type in enumerate(map_types, 1):
    y = approximate_data[map_type]
    label = map_type
    label = r"$\mathtt{{approximate\ {0}}}$".format(label.replace("_", "\_"))
    axs.plot(x, y, label=label, color=material_palette[(i + 1) ** 2 % 16])

    y = data[map_type]
    label = map_type
    label = r"$\mathtt{{exact\ {0}}}$".format(label.replace("_", "\_"))
    axs.plot(x, y, label=label, color=material_palette[(i + 2) ** 2 % 16])

axs.set_xlim(0, 4)
axs.set_xlabel("$\mathrm{Atomic\ distance\ (\AA)}$")

axs.set_ylim(-8, -4.0)
axs.set_ylabel("Energy")
axs.set_yticks([y for y in np.arange(-8, -3.5, 0.5)])

subtitle = ", ".join(
    [
        file_name.replace("_", "\_"),
        backend.backend_name.replace("_", "\_"),
        device.name().replace("_", "\_"),
    ]
)
axs.set_title("$\mathrm{{Energy\ for\ LiH}}$\n\n$\mathtt{{{0}}}$".format(subtitle))

axs.legend(loc=1, title=r"$\mathbf{{Qubit\ mappings}}$")
axs.grid(linestyle="-")
fig.tight_layout()
fig.savefig(f"{file_name}__energy.jpg", dpi=600)
plt.show()
plt.close(fig)

In [ ]:
minimum_idxs = data[map_types].idxmin()
minimun_energies = data[map_types].min()
minimun_distances = pandas.Series(
    {map_type: data["distance"][minimum_idxs[map_type]] for map_type in map_types}
)

In [ ]:
ratio = 16.0 / 9.0
width = 16
height = width / ratio

fig = plt.figure(1, figsize=(width, height), frameon=False)
axs = fig.add_subplot("111")
x_upper_lim = 0

for i, map_type in enumerate(map_types):
    key = (minimun_distances[map_type], map_type)

    x = intermediate_results[key]["counts"]
    y = intermediate_results[key]["values"]

    label = map_type
    label = r"$\mathtt{{{0}}}$".format(label.replace("_", "\_"))
    axs.plot(
        x,
        y,
        linestyle="dashed",
        marker="*",
        label=label,
        color=material_palette[(i + 1) ** 2 % 16],
    )

    if max(x) > x_upper_lim:
        x_upper_lim = max(x)

x_upper_lim = x_upper_lim + 10
x_upper_lim = x_upper_lim - (x_upper_lim % 10)

axs.set_ylim(-1.5, 1.5)
axs.set_ylabel("Energy Minimization")
axs.set_yticks(np.arange(-1.5, 2, 0.5))

axs.set_xlim(0, x_upper_lim)
axs.set_xlabel("$\mathrm{Eval.\ counts}$")

subtitle = ", ".join(
    [
        file_name.replace("_", "\_"),
        backend.backend_name.replace("_", "\_"),
        device.name().replace("_", "\_"),
    ]
)
axs.set_title(
    "$\mathrm{{Energy\ convergence\ for\ VQE\ procedure\ {0}}}$\n\n$\mathtt{{{1}}}$".format(
        variational_form_kind.upper(), subtitle
    )
)

axs.legend(loc=1, title=r"$\mathbf{{Qubit\ mappings}}$")
axs.grid(linestyle="-")
fig.tight_layout()
fig.savefig(f"{file_name}__energy_convergence_at_minimum_energy.jpg", dpi=600)
plt.show()
plt.close(fig)

In [ ]:
ratio = 16.0 / 9.0
width = 16
height = width / ratio

fig = plt.figure(1, figsize=(width, height), frameon=False)
axs = fig.add_subplot("111")
x_upper_lim = 0

for i, map_type in enumerate(map_types):
    key = (minimun_distances[map_type], map_type)

    exact_energy_reference = exact_energies[key]
    exact_energy_reference = exact_energy_reference["eigenvalues"][0].real

    x = intermediate_results[key]["counts"]
    y = np.array(intermediate_results[key]["values"]) - exact_energy_reference
    y = abs(y / exact_energy_reference)

    label = map_type
    label = r"$\mathtt{{{0}}}$".format(label.replace("_", "\_"))
    axs.plot(
        x,
        y,
        linestyle="dashed",
        marker="*",
        label=label,
        color=material_palette[(i + 1) ** 2 % 16],
    )

    if max(x) > x_upper_lim:
        x_upper_lim = max(x)

x_upper_lim = x_upper_lim + 10
x_upper_lim = x_upper_lim - (x_upper_lim % 10)

axs.set_ylabel("Logscale\ Error")
axs.set_yscale("log")

axs.set_xlim(0, x_upper_lim)
axs.set_xlabel("$\mathrm{Eval.\ counts}$")

subtitle = ", ".join(
    [
        file_name.replace("_", "\_"),
        backend.backend_name.replace("_", "\_"),
        device.name().replace("_", "\_"),
    ]
)
axs.set_title(
    "$\mathrm{{Logscale\ Error\ of\ VQE\ procedure\ {0}}}$\n\n$\mathtt{{{1}}}$".format(
        variational_form_kind.upper(), subtitle
    )
)

axs.legend(loc=1, title=r"$\mathbf{{Qubit\ mappings}}$")
axs.grid(linestyle="-")
fig.tight_layout()
fig.savefig(f"{file_name}__logscale_error_at_minimum_energy.jpg", dpi=600)
plt.show()
plt.close(fig)